## RNN architechture

In [156]:
%matplotlib inline
from gensim import corpora, models, similarities
from gensim import corpora
from gensim.models import word2vec
import gensim
import pandas as pd
import numpy as np
import os
from glob import glob
from collections import Counter
import matplotlib.pyplot as plt
import re

In [157]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
os.chdir('/Users/xiaodiu/Documents/github/JudgeSentences/train/')
from train_ngrams import textfeature

In [197]:
from keras.models import model_from_json
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model
from keras.layers import Lambda
from keras import initializations
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
import pickle
from nltk import tokenize
import tensorflow as tf
sess = tf.InteractiveSession()

### Data prepare

In [99]:
case_train = pd.read_csv('../holger_train_disagg_final.csv',index_col= 0)
case_test  = pd.read_csv('../holger_test_disagg_final.csv' ,index_col= 0)
case_train.head(1)

,sentyr,trial,monsex,nocounts,judgeid,judge_yearsonbench,crime,demean_harshness,race1,race2,...,race6,race7,race8,gender,x_black,x_dem,party,age,aba_best,christian
118870,2006,0,0.0,1.0,"WALTER, JOHN F.",4,12.0,-0.126023,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,62.276836,1.0,0.0


In [116]:
case_train_for_rnn = case_train.loc[:,['sentyr','judgeid','crime','demean_harshness']]
case_train_for_rnn = pd.pivot_table(case_train_for_rnn, values='demean_harshness', index=['sentyr', 'judgeid'],
                                    columns=['crime'], aggfunc=np.mean, fill_value  = 0)
case_train_for_rnn = pd.DataFrame(case_train_for_rnn.to_records())

case_test_for_rnn = case_test.loc[:,['sentyr','judgeid','crime','demean_harshness']]
case_test_for_rnn = pd.pivot_table(case_test_for_rnn, values='demean_harshness', index=['sentyr', 'judgeid'],
                                    columns=['crime'], aggfunc=np.mean, fill_value  = 0)
case_test_for_rnn = pd.DataFrame(case_test_for_rnn.to_records())
print(case_test_for_rnn.shape)
print(case_train_for_rnn.shape)
case_train_for_rnn.head(1)

(4358, 24)
(4840, 24)


,sentyr,judgeid,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,...,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0
0,2002,"BABCOCK, LEWIS T.",0.0,0.0,0.0,0.0,0.0,0.0,0.612752,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Train and test data

In [122]:
judge_year_case = pd.read_pickle('../datasets/judge2year_case.pkl')
opinion2floder = pd.read_pickle('../datasets/opinion2floder.pkl')
judges = judge_year_case.keys()
data_train = case_train_for_rnn
data_test = case_test_for_rnn

In [182]:
y_train = []
y_test = []
texts = []
max_year = 0
max_judge = 0
reviews = Counter()

In [124]:
EMBEDDING_DIM = 100
MAX_SENTS = 50
MAX_SENT_LEN = 30
MAX_NB_WORDS = 500000

In [ ]:
year_range = (2002,2012)
for i, judge in enumerate(judges):
    opinions = judge_year_case[judge]
    review_year = Counter()
    for year in opinions.keys():
        review_all = []
        if year < year_range[0] or year > year_range[1]:
            continue
        opilist = opinions[year]
        for opi in opilist:    
            fold = opinion2floder[str(opi)]
            filepath = '../rawdata/text/' + fold + '/' + str(opi) + '.txt'
            #print(filepath)
            text = open(filepath).read()    
            normtext = text.strip().lower()
            sentences = tokenize.sent_tokenize(normtext)
            for sent in sentences:
                if len(sent.split(' ')) > 5:
                    sent = re.sub('[^a-z]',' ',sent)
                    review_all.append(sent)
                    texts.append(sent)
        review_year[year] = review_all
    reviews[judge] = review_year
    print(i)

In [128]:
pd.to_pickle(reviews, '../datasets/lstmdata/all_reviews.pkl')
pd.to_pickle(texts, '../datasets/lstmdata/textsfortoken.pkl')

In [186]:
reviews = pd.read_pickle('../datasets/lstmdata/all_reviews.pkl')

In [187]:
train_len = 0
for i in data_train.index:
    senty = data_train.sentyr[i]
    judge = data_train.judgeid[i]
    try :
        opinions = reviews[judge][senty]
        if opinions == 0:
            print('no opnions this %s for %s' %(senty, judge))
            continue
        train_len += 1
    except TypeError :
        print('no opnions this %s for %s' %(senty, judge))
    
test_len = 0
for i in data_test.index:
    senty = data_test.sentyr[i]
    judge = data_test.judgeid[i]
    try :
        opinions = reviews[judge][senty]
        if opinions == 0:
            print('no opnions this %s for %s' %(senty, judge))
            continue
        test_len += 1
    except TypeError :
        print('no opnions this %s for %s' %(senty, judge))

no opnions this 2002 for CALDWELL, KAREN K.
no opnions this 2002 for CONWAY, JOHN E.
no opnions this 2002 for DAWSON, KENT J.
no opnions this 2002 for DOWNES, WILLIAM F.
no opnions this 2002 for JARVIS, JAMES H., II
no opnions this 2002 for MCKINLEY, JOSEPH H., JR.
no opnions this 2002 for PALLMEYER, REBECCA R.
no opnions this 2002 for PHILLIPS, VIRGINIA A.
no opnions this 2002 for SMITH, WALTER S., JR.
no opnions this 2002 for ZAPATA, FRANK R.
no opnions this 2003 for ADAMS, JOHN R.
no opnions this 2003 for ALDRICH, ANN
no opnions this 2003 for ALTONAGA, CECILIA M.
no opnions this 2003 for ANDERSON, JOSEPH F., JR.
no opnions this 2003 for ANDERSON, PERCY
no opnions this 2003 for BARBIER, CARL J.
no opnions this 2003 for BEISTLINE, RALPH R.
no opnions this 2003 for BRACK, ROBERT C.
no opnions this 2003 for BROWNING, JAMES O.
no opnions this 2003 for BURGESS, FRANKLIN D.
no opnions this 2003 for CALDWELL, KAREN K.
no opnions this 2003 for CEBULL, RICHARD F.
no opnions this 2003 for CLEV

In [144]:
texts = pd.read_pickle('../datasets/lstmdata/textsfortoken.pkl')
tokenizer = Tokenizer(nb_words=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index

In [145]:
filep = open('../datasets/lstmdata/tokenizer.out', 'wb')
pickle.dump(tokenizer, filep)

In [188]:
lstm_train = np.zeros((train_len,MAX_SENTS, MAX_SENT_LEN), dtype='int32')
lstm_test = np.zeros((test_len,MAX_SENTS, MAX_SENT_LEN), dtype='int32')

In [189]:
num = 0
for i in data_train.index:
    senty = data_train.sentyr[i]
    judge = data_train.judgeid[i]
    try :
        opinions = reviews[judge][senty]
        if opinions == 0:
            print('no opnions this %s for %s' %(senty, judge))
            continue
        y_train.append(list(data_train.loc[i, '1.0':'22.0' ]))
        for j, sent in enumerate(opinions):
            if j< MAX_SENTS:
                sent = re.sub('[^a-z]',' ',sent)
                wordTokens = text_to_word_sequence(sent)
                k=0
                for _, word in enumerate(wordTokens):
                    if k<MAX_SENT_LEN:
                        index = tokenizer.word_index[word]
                        if index == 0:
                            continue
                        lstm_train[num,j,k] = index
                        #lstm_train[num,j,k] = tokenizer.word_index[word]
                        k=k+1  
        num += 1
    except TypeError :
        print('no opnions this %s for %s' %(senty, judge)) 

num = 0
for i in data_test.index:
    senty = data_test.sentyr[i]
    judge = data_test.judgeid[i]
    try :
        opinions = reviews[judge][senty]
        if opinions == 0:
            print('no opnions this %s for %s' %(senty, judge))
            continue
        y_test.append(list(data_test.loc[i, '1.0':'22.0' ]))
        for j, sent in enumerate(opinions):
            if j< MAX_SENTS:
                sent = re.sub('[^a-z]',' ',sent)
                wordTokens = text_to_word_sequence(sent)
                k=0
                for _, word in enumerate(wordTokens):
                    if k<MAX_SENT_LEN :
                        index = tokenizer.word_index[word]
                        if index == 0:
                            continue
                        lstm_test[num,j,k] = index
                        k=k+1 
        num += 1
    except TypeError :
        print('no opnions this %s for %s' %(senty, judge)) 

no opnions this 2002 for CALDWELL, KAREN K.
no opnions this 2002 for CONWAY, JOHN E.
no opnions this 2002 for DAWSON, KENT J.
no opnions this 2002 for DOWNES, WILLIAM F.
no opnions this 2002 for JARVIS, JAMES H., II
no opnions this 2002 for MCKINLEY, JOSEPH H., JR.
no opnions this 2002 for PALLMEYER, REBECCA R.
no opnions this 2002 for PHILLIPS, VIRGINIA A.
no opnions this 2002 for SMITH, WALTER S., JR.
no opnions this 2002 for ZAPATA, FRANK R.
no opnions this 2003 for ADAMS, JOHN R.
no opnions this 2003 for ALDRICH, ANN
no opnions this 2003 for ALTONAGA, CECILIA M.
no opnions this 2003 for ANDERSON, JOSEPH F., JR.
no opnions this 2003 for ANDERSON, PERCY
no opnions this 2003 for BARBIER, CARL J.
no opnions this 2003 for BEISTLINE, RALPH R.
no opnions this 2003 for BRACK, ROBERT C.
no opnions this 2003 for BROWNING, JAMES O.
no opnions this 2003 for BURGESS, FRANKLIN D.
no opnions this 2003 for CALDWELL, KAREN K.
no opnions this 2003 for CEBULL, RICHARD F.
no opnions this 2003 for CLEV

In [195]:
np.save("../datasets/lstmdata/lstm_train.npy", lstm_train)
np.save("../datasets/lstmdata/lstm_test.npy", lstm_test)
np.save("../datasets/lstmdata/y_train.npy", y_train)
np.save("../datasets/lstmdata/y_test.npy", y_test)

## Attention model

In [221]:
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        self.W = self.init([input_shape[-1]])
        self.b = self.init([1])
        #self.W = tf.contrib.keras.initializers.RandomNormal()([input_shape[-1]])
        #self.b = tf.contrib.keras.initializers.RandomNormal()([1])
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W, self.b]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(x * self.W + self.b)
        
        ai = K.exp(eij)
        weights = ai/ tf.expand_dims(K.sum(ai, axis=1),1)
        
        weighted_input = x*weights
        return tf.reduce_sum(weighted_input ,axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [220]:
epsilon = 1.0e-9
def vector_loss(y_true, y_pred):
    '''Just another crossentropy'''
    index = 1.0 - tf.cast(tf.equal(y_true, 0.) , tf.float32)
    num = tf.reduce_sum(index) * 1.0 + 0.00001
    return tf.reduce_sum((y_true * index - y_pred * index)**2) / num

In [161]:
lstm_train = np.load("../datasets/lstmdata/lstm_train.npy")
lstm_test = np.load("../datasets/lstmdata/lstm_test.npy")
y_train = np.load("../datasets/lstmdata/y_train.npy")
y_test = np.load("../datasets/lstmdata/y_test.npy")

In [225]:
EMBEDDING_DIM = 100
MAX_SENTS = 50
MAX_SENT_LEN = 30
MAX_NB_WORDS = 500000

In [226]:
tokenizer = pickle.load( open('../datasets/lstmdata/tokenizer.out', 'rb'))
word_index = tokenizer.word_index
filepath="../datasets/lstmdata/weights.best.hdf5"

In [227]:
model = word2vec.Word2Vec.load('../datasets/word2vec.model')
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = model.wv[word].tolist()
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    except KeyError as err:
        embedding_matrix[i] = np.zeros(EMBEDDING_DIM)
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LEN,
                            trainable=True)

In [229]:
sentence_input = Input(shape=(MAX_SENT_LEN,), dtype='int32')
#sentence_input = Lambda(lambda x: x[:,36:], output_shape=(MAX_SENT_LEN,))(data_input)
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(200))(l_lstm)
l_att = AttLayer()(l_dense)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LEN), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_dense_sent = TimeDistributed(Dense(200))(l_lstm_sent)
l_att_sent = AttLayer()(l_dense_sent)
preds = Dense(22, activation=None)(l_att_sent)
model = Model(review_input, preds)

model.compile(loss= vector_loss,
              optimizer='adam',
              metrics=['mse','mae'])


In [230]:
checkpoint = ModelCheckpoint(filepath, monitor='val_mse', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

print("model fitting - Hierachical attention network")
history =model.fit(lstm_train, y_train, validation_data=(lstm_test, y_test),
        nb_epoch = 1, batch_size=1, callbacks=callbacks_list, verbose=1)

filep = open('../datasets/lstmdata/history.out', 'wb')
pickle.dump(tokenizer, filep)

model_json = model.to_json()
with open("../datasets/lstmdata/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../datasets/lstmdata/model.h5")

model fitting - Hierachical attention network
Train on 4157 samples, validate on 3797 samples
Epoch 1/1
 174/4157 [>.............................] - ETA: 2750s - loss: 0.1460 - mean_squared_error: 0.1318 - mean_absolute_error: 0.2592 

KeyboardInterrupt: 

## Visualization